In [ ]:
# Required Packages (Add under Advanced Options > Packages):

'''
org.apache.hadoop:hadoop-azure:3.3.2
org.apache.hadoop:hadoop-azure-datalake:3.3.2
'''

In [ ]:
# Use a cluster of atleast 32 GiB storage and 300 GB scratch disk to run the last query without issues

In [ ]:
# Azure ADLS configuration for accessing nyc-taxi-data public dataset from Azure (no credentials needed)
# https://learn.microsoft.com/en-us/azure/open-datasets/dataset-taxi-yellow?tabs=azureml-opendatasets

blob_account_name = "azureopendatastorage"
blob_container_name = "nyctlc"
blob_relative_path = "yellow"
blob_sas_token = r""
wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path)
spark.conf.set('fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name), blob_sas_token)

# Read data from ADLS
df = spark.read.parquet(wasbs_path)
df.createOrReplaceTempView("source")

In [ ]:
#Simple Query

result = spark.sql("""
SELECT *
FROM source
ORDER BY tpepPickupDateTime DESC
LIMIT 1000
""")
result.show()

In [ ]:
# Medium Query

display(spark.sql("""SELECT puYear, puMonth, totalAmount
FROM (
SELECT puYear, puMonth, totalAmount, ROW_NUMBER() OVER (partition by puYear, puMonth order by totalAmount) as rn
FROM source
) ranked
WHERE ranked.rn = 1""").show())

In [ ]:
# Selective Query

display(spark.sql("""SELECT *
  FROM source
  WHERE startLon BETWEEN '-73.97' AND '-73.93'
  AND startLat BETWEEN '40.75' AND '40.79'
  AND vendorID = 'CMT'
  AND tpepPickupDateTime BETWEEN '2014-04-30 23:55:04.000' and '2014-06-30 23:55:04.000'
  and paymentType != 'CSH'
  AND totalAmount < '4.00'
  AND endLon < '-74'""").show())

In [ ]:
# Ridiculous Query

columns = spark.table("source").columns

group_by_query = f"""
SELECT COUNT(*) AS total_rows
FROM (
  SELECT *
  FROM source
  GROUP BY {', '.join(columns)}
) a
"""
# Execute and display the query
display(spark.sql(group_by_query).show())